In [ ]:
import os
os.environ['CUDA_VISIBLE_DEVICES'] = '3'

In [ ]:
import random
import json

In [ ]:
from transformers import AutoModelForCausalLM, AutoTokenizer

device = "cuda" # the device to load the model onto

model = AutoModelForCausalLM.from_pretrained("mistralai/Mistral-7B-Instruct-v0.2")
tokenizer = AutoTokenizer.from_pretrained("mistralai/Mistral-7B-Instruct-v0.2")


model.to(device)

In [ ]:
messages = [
    {"role": "user", "content": "You are a helpful abiding assistant. Your inputs will consist of a question and a correct answer, and a answer from a model. Your response should start with a single word either a yes if the model's answer means the correct answer, else the starting word should be no."},
    {"role": "assistant", "content": "Okay, I will do that."},
    {"role": "user", "content": "'Question: For which of these two scenarios does the main character (who uses I/me/my) do something clearly morally wrong, according to ordinary moral standards in the US as of 2020? Scenario 1 | I bared my entire naked body at the beach. Scenario 2 | I told her it was safe to jump off the couch. Ground Truth: Wrong, Not wrong. Model Answer: Not wrong, Wrong"}
]

encodeds = tokenizer.apply_chat_template(messages, return_tensors="pt")

model_inputs = encodeds.to(device)

generated_ids = model.generate(model_inputs, max_new_tokens=1, do_sample=False)
decoded = tokenizer.batch_decode(generated_ids)
#print(decoded[0])
tokenizer.batch_decode(generated_ids[:, model_inputs.size(1):])[0]

In [ ]:
generated_ids.shape, model_inputs.shape, len(decoded[0]), len(tokenizer.batch_decode(model_inputs)[0]), generated_ids[:, model_inputs.size(1):].shape, len(tokenizer.batch_decode(generated_ids[:, model_inputs.size(1):])[0]), tokenizer.batch_decode(generated_ids[:, model_inputs.size(1):])[0]

In [ ]:
decoded[0][-3:]

In [ ]:
model.generation_config

In [ ]:
all_data = []
with open('perturbed1.txt', 'r') as f:
    all_data = f.readlines()
with open('wrong.txt', 'r') as f:
    all_data += f.readlines()
print(len(all_data))
random.shuffle(all_data)

In [ ]:
def form_prompt(question, correct_ans, model_ans):
    return f"Question: {question} Ground Truth: {correct_ans}. Model Answer: {model_ans}"

In [ ]:
all_data[1]

In [ ]:
correct = 0
total = 0
diff = 0
errors = []
for i, data in enumerate(all_data):
    try:
        if i % 100 == 1:
            print(correct/(total+1e-3), correct, total)
        data = json.loads(data)
        messages = [
            {"role": "user", "content": "context: You are a helpful abiding assistant. Your inputs will consist of a question and a correct answer, and a answer from a model. Your response should start with a single word either a yes if the model's answer means the correct answer, else the starting word should be no."},
            {"role": "assistant", "content": "Okay, I will do that."},
        ]
        expected_checker_ans = 'yes'
        if 'perturbed_answer' in data:
            messages.append({"role": "user", "content": form_prompt(data['question'], correct_ans=data['correct_answer'], model_ans=data['perturbed_answer'])})
        else:
            messages.append({"role": "user", "content": form_prompt(question=data['question'], correct_ans=data['correct_answer'], model_ans=data['wrong_answer'])})
            expected_checker_ans = 'no'
        # print(messages)
        encodeds = tokenizer.apply_chat_template(messages, return_tensors="pt")

        model_inputs = encodeds.to(device)

        generated_ids = model.generate(model_inputs, max_new_tokens=1, do_sample=False, pad_token_id=tokenizer.eos_token_id)
        # print(tokenizer.batch_decode(generated_ids))
        decoded = tokenizer.batch_decode(generated_ids[:, model_inputs.size(1):])
        decoded = str(decoded[0]).strip().lower().strip()
        # print(decoded, expected_checker_ans)
        if expected_checker_ans == decoded:
            correct += 1
        else:
            if decoded != 'yes' and decoded != 'no':
                diff += 1
                with open('diff.txt', 'a') as f:
                    f.write(str(data)+','+str(decoded)+'\n')
        total += 1
    except Exception as e:
        print(correct/total, correct, total)
        error = e
        errors.append(data)
        continue
correct/total, correct, total, errors